<a href="https://colab.research.google.com/github/urvog/incomesalary/blob/main/Unidad_II_Tema_15_Random_Forest%2C_XGBoost%2C_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tema 15,16 ALGORITMOS BAGGING, BOOSTING, NEURAL NETWORKS

<img src="https://www.neuraldojo.org/media/forest4.jpeg"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" width="100%"/>
     
##Ingresos $ de personas adultas

**Objetivo**
Predecir si los ingresos exceden los $50K/año según los datos del censo. También conocido como conjunto de datos de "Ingresos del censo".

>50K, <=50K.

Columnas:
- age: edad (continuous)
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, - State-gov, Without-pay, Never-worked.
- fnlwgt: continuous.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18030331028342760550
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16154099712
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17246102151484371260
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
 xla_global_id: 416903419]

## 1.- Importando los datos

In [ ]:
!wget https://neuraldojo.org/media/income/income.csv

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
df_income = pd.read_csv('income.csv')

In [ ]:
df_income.shape

##2.- Analysis Exploratorio de Datos (EDA)

In [ ]:
df_income.info()

In [ ]:
df_income.isnull().sum()

In [ ]:
df_income.columns

In [ ]:
#Verificamos la distribución de nuestra variable objetivo
df_income['income_>50K'].value_counts()

### 2.1 Analisis Univariable

In [ ]:
# Distribucion de la Edad
age = df_income['age'].value_counts()

plt.figure(figsize=(10, 5))
plt.style.use('fivethirtyeight')
sns.distplot(df_income['age'], bins=20)
plt.title('Distribucion de la edad', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Edad', fontdict={'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.show()

In [ ]:
# Distribucion de la educacion
edu = df_income['education'].value_counts()

plt.style.use('seaborn')
plt.figure(figsize=(10, 5))
sns.barplot(edu.values, edu.index, palette='Paired')
plt.title('Distribucion de la Educacion', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Numero de personasl', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Educacion', fontdict={'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
# Distribuion de los años de educación
edu_num = df_income['educational-num'].value_counts()

plt.style.use('ggplot')
plt.figure(figsize=(10, 5))
sns.barplot(edu_num.index, edu_num.values, palette='colorblind')
plt.title('Distribucion de los años de educación', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Años', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de Personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=12)
plt.show()

In [ ]:
# Una torta de distribucion del estado scivil
marital = df_income['marital-status'].value_counts()

plt.style.use('default')
plt.figure(figsize=(7, 5))
plt.pie(marital.values, labels=marital.index, startangle=10, explode=(
    0, 0.20, 0, 0, 0, 0, 0), shadow=True, autopct='%1.1f%%')
plt.title('Distribucion del estado civil', fontdict={
          'fontname': 'Monospace', 'fontsize': 15, 'fontweight': 'bold'})
plt.legend()
plt.legend(prop={'size': 7})
plt.axis('equal')
plt.show()

In [ ]:
# Dibujemos un dona del tipo de relación
relation = df_income['relationship'].value_counts()

plt.style.use('bmh')
plt.figure(figsize=(10, 5))
plt.pie(relation.values, labels=relation.index,
        startangle=50, autopct='%1.1f%%')
centre_circle = plt.Circle((0, 0), 0.7, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Distribucion de la Relación', fontdict={
          'fontname': 'Monospace', 'fontsize': 15, 'fontweight': 'bold'})
plt.axis('equal')
plt.legend(prop={'size': 10})
plt.show()

In [ ]:
# Distribucion del Sexo
sex = df_income['gender'].value_counts()

plt.style.use('default')
plt.figure(figsize=(7, 5))
sns.barplot(sex.index, sex.values)
plt.title('Distribucion de Sexo', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Sexo', fontdict={'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.grid()
plt.show()

In [ ]:
# Creando un mapa de arbol para la razas
!pip install squarify
import squarify
race = df_income['race'].value_counts()

plt.style.use('default')
plt.figure(figsize=(7, 5))
squarify.plot(sizes=race.values, label=race.index, value=race.values)
plt.title('Distribucion de la Raza', fontdict={
          'fontname': 'Monospace', 'fontsize': 15, 'fontweight': 'bold'})
plt.show()

In [ ]:
# Creando una distribución de las horas por semana
hours = df_income['hours-per-week'].value_counts().head(10)

plt.style.use('bmh')
plt.figure(figsize=(12, 5))
sns.barplot(hours.index, hours.values, palette='colorblind')
plt.title('Distribution of Hours of work per week', fontdict={
          'fontname': 'Monospace', 'fontsize': 15, 'fontweight': 'bold'})
plt.xlabel('Horas a la semana', fontdict={'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=12)
plt.show()

###2.2 Analisis Bi-variable

In [ ]:
# Ingresos vs Edad
plt.style.use('default')
plt.figure(figsize=(15, 7))
sns.countplot(df_income['age'], hue=df_income['income_>50K'])
plt.title('Distribucion Ingresos vs Edad', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Edad', fontdict={'fontname': 'Monospace', 'fontsize': 10})
plt.ylabel('Numero de Personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 10})
plt.tick_params(labelsize=10)
plt.legend(loc=1, prop={'size': 15})
plt.show()

In [ ]:
# Ingresos vs Edacacion
plt.style.use('seaborn')
plt.figure(figsize=(15, 7))
sns.countplot(df_income['education'],
              hue=df_income['income_>50K'], palette='colorblind')
plt.title('Distribution INgresos vs Education', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Educacion', fontdict={'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.legend(loc=1, prop={'size': 10})
plt.show()

In [ ]:
# Ingresos vs Años de Educación
plt.style.use('bmh')
plt.figure(figsize=(15, 7))
sns.countplot(df_income['educational-num'],
              hue=df_income['income_>50K'])
plt.title('Ingresos Vs Años de Educación', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Años de Educación', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de Personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.legend(loc=1, prop={'size': 15})
plt.show()

In [ ]:
# Ingresos vs Estado Civil
plt.style.use('seaborn')
plt.figure(figsize=(15, 7))
sns.countplot(df_income['marital-status'], hue=df_income['income_>50K'])
plt.title('Ingresos Vs Estado Civil', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Estado Civil', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.legend(loc=1, prop={'size': 15})
plt.show()

In [ ]:
# Ingresos vs Raza
plt.style.use('fivethirtyeight')
plt.figure(figsize=(15, 7))
sns.countplot(df_income['race'], hue=df_income['income_>50K'])
plt.title('Ingresos vs Raza', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Raza', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de Personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.legend(loc=1, prop={'size': 15})
plt.show()

In [ ]:
# Ingresos vs Genero
plt.style.use('fivethirtyeight')
plt.figure(figsize=(7, 3))
sns.countplot(df_income['gender'], hue=df_income['income_>50K'])
plt.title('Ingresos Vs Genero', fontdict={
          'fontname': 'Monospace', 'fontsize': 20, 'fontweight': 'bold'})
plt.xlabel('Sexo', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.ylabel('Numero de Personas', fontdict={
           'fontname': 'Monospace', 'fontsize': 15})
plt.tick_params(labelsize=10)
plt.legend(loc=1, prop={'size': 10})
plt.show()

###2.3 Analisis Multivariable


In [ ]:
#Mapa de calor de correlaciones
sns.heatmap(df_income.corr(), annot= True)

In [ ]:
#Pairplot
sns.pairplot(df_income)

### 2.3 Manipulacion de los datos

In [ ]:
#Label Encoder
#A diferencia del método get_dummies(), label encoder convierte las columnas categoricas en valor númerico
from sklearn.preprocessing import LabelEncoder

for col in df_income.columns:
    if df_income[col].dtypes == 'object':
        encoder = LabelEncoder()
        df_income[col] = encoder.fit_transform(df_income[col])

In [ ]:
df_income.head()

In [ ]:
#Dividimos nuestros datos para entrenar el modelo (Train, Validation) y para Test
df_train_ = df_income.sample(frac= .80)
df_test = df_income.drop(df_train_.index)

In [ ]:
#Dividimos nuesros datos train, validation and test
X = df_train_.drop('income_>50K', axis = 1)
y = df_train_['income_>50K']

X_test = df_test.drop('income_>50K', axis = 1)
y_test = df_test['income_>50K']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0, test_size = 0.3)


#Random Forest

##3.- Modelado ML

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(random_state = 0)
clf_rf.fit(X_train, y_train)

##4.- Evaluamos

In [ ]:
#Imprimimos Scores
print("Train Score:", clf_rf.score(X_train,y_train))
print("Validation Score:", clf_rf.score(X_valid,y_valid))
print("Test Score:", clf_rf.score(X_test,y_test))

##5.- Ajustamos Parametros

- n_estimators: número de árboles que construye el algoritmo antes de promediar las predicciones.

- max_features: número máximo de características que el bosque aleatorio considera dividir un nodo.

- min_sample_leaf: determina el número mínimo de hojas necesarias para dividir un nodo interno.

In [ ]:
clf_rf_ = RandomForestClassifier(random_state = 0, max_depth=20,n_estimators=100)
clf_rf_.fit(X_train, y_train)

#Imprimimos Scores
print("Train Score:", clf_rf_.score(X_train,y_train))
print("Validation Score:", clf_rf_.score(X_valid,y_valid))
print("Test Score:", clf_rf_.score(X_test,y_test))

###5.1 Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
clf_rf_ = RandomForestClassifier(random_state = 0)
model_params = {
    'max_depth': [5,10,15,20,50],
    'n_estimators':[100,200,300],
    'min_samples_split': [5,10,20],
}

clf_grid = GridSearchCV(clf_rf_,model_params, cv = 3, verbose=10)
clf_grid.fit(X_train,y_train)


In [ ]:
print(clf_grid.best_estimator_.get_params())

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

clf_rf_best = RandomForestClassifier(criterion ='gini', max_depth = 20, max_features='auto', min_samples_leaf=1, min_samples_split = 20, n_estimators = 200)
clf_rf_best.fit(X_train,y_train)

#Imprimimos Scores Acuracy
print("Train Score Grid Search (Accuracy):", clf_rf_best.score(X_train,y_train))
print("Validation Grid Search Score (Accuracy):", clf_rf_best.score(X_valid,y_valid))
print("Test Score Grid Search (Accuracy):", clf_rf_best.score(X_test,y_test))

#Imprimimos Scores F1-Score
y_pred_val = clf_rf_best.predict(X_valid)
y_pred_test = clf_rf_best.predict(X_test)
#f1_score(Y_test, Y_pred_log_reg))

print("Validation Grid Search Score (F1-Score):", f1_score(y_valid,y_pred_val))
print("Test Grid Search Score(F1-Score):", f1_score(y_test,y_pred_test))

#XGBoost

<img src="https://www.neuraldojo.org/media/boost3.jpeg"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" width="100%"/>

##3.- Modelado ML

In [ ]:
from xgboost import XGBClassifier
clf_xgb = XGBClassifier(random_state = 0)
clf_xgb.fit(X_train, y_train)

##4.- Evaluamos

In [ ]:
print("Train Score:", clf_xgb.score(X_train,y_train))
print("Validation Score:", clf_xgb.score(X_valid,y_valid))
print("Test Score:", clf_xgb.score(X_test,y_test))

##5.- Ajustamos Parámetros

In [ ]:
#Creamos nuestro modelo basico
clf_xgb_ = XGBClassifier(random_state = 0)

#imprimimos los parametros disponibles
clf_xgb_.get_params()

In [ ]:
model_params = {
    'learning_rate': [0.05,0.1],
    'max_depth': [5,10,15],
    'n_estimators':[100,200]
}
clf_grid_xgb = GridSearchCV(clf_xgb_,model_params, cv = 5, verbose=10)
clf_grid_xgb.fit(X_train,y_train)

In [ ]:
print(clf_grid_xgb.best_estimator_.get_params())

In [ ]:
clf_xgb_best = XGBClassifier(random_state = 0, max_depth = 5, n_estimators = 200)
clf_xgb_best.fit(X_train,y_train)

#Imprimimos Scores Acuracy
print("Train Score Grid Search (Accuracy):", clf_xgb_best.score(X_train,y_train))
print("Validation Grid Search Score (Accuracy):", clf_xgb_best.score(X_valid,y_valid))
print("Test Score Grid Search (Accuracy):", clf_xgb_best.score(X_test,y_test))

#Imprimimos Scores F1-Score
y_pred_val = clf_xgb_best.predict(X_valid)
y_pred_test = clf_xgb_best.predict(X_test)
#f1_score(Y_test, Y_pred_log_reg))

print("Validation Grid Search Score (F1-Score):", f1_score(y_valid,y_pred_val))
print("Test Grid Search Score(F1-Score):", f1_score(y_test,y_pred_test))

# Neural Networks

<img src="https://www.neuraldojo.org/media/nn.jpg"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" width="100%"/>

In [ ]:
#normalizamos los datos, en redes neuronales es recomendable realizar este paso de reescalado de los datos
from sklearn.preprocessing import MinMaxScaler

columns = X_train.columns

scaler = MinMaxScaler()

X_train_norm = pd.DataFrame(scaler.fit_transform(X_train), columns = columns)
X_valid_norm = pd.DataFrame(scaler.fit_transform(X_valid), columns = columns)
X_test_norm = pd.DataFrame(scaler.fit_transform(X_test), columns = columns)

X_train_norm.head()

##3.- Modeloado ML

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_nn = MLPClassifier(random_state = 0, hidden_layer_sizes = (100,50,20,10,5))
clf_nn.fit(X_train_norm, y_train)

In [ ]:
clf_nn.get_params()

##4.- Evaluamos

In [ ]:
print("Train Score:", clf_nn.score(X_train,y_train))
print("Validation Score:", clf_nn.score(X_valid,y_valid))
print("Test Score:", clf_nn.score(X_test,y_test))

##5.- Ajustamos parámetros

In [ ]:
#Creamos nuestro modelo basico
clf_nn_ = MLPClassifier(random_state = 0)

#imprimimos los parametros disponibles
clf_nn_.get_params()

In [ ]:
clf_nn_ = MLPClassifier(random_state = 0)
model_params = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [ ]:
clf_grid_nn = GridSearchCV(clf_nn_,model_params, cv = 5, verbose=10)
clf_grid_nn.fit(X_train,y_train)

In [ ]:
clf_grid_nn.best_estimator_.get_params()

In [ ]:
clf_nn_best = MLPClassifier()
clf_nn_best.fit(X_train,y_train)

#Imprimimos Scores Acuracy
print("Train Score Grid Search (Accuracy):", clf_nn_best.score(X_train,y_train))
print("Validation Grid Search Score (Accuracy):", clf_nn_best.score(X_valid,y_valid))
print("Test Score Grid Search (Accuracy):", clf_nn_best.score(X_test,y_test))

#Imprimimos Scores F1-Score
y_pred_val = clf_nn_best.predict(X_valid)
y_pred_test = clf_nn_best.predict(X_test)
#f1_score(Y_test, Y_pred_log_reg))

print("Validation Grid Search Score (F1-Score):", f1_score(y_valid,y_pred_val))
print("Test Grid Search Score(F1-Score):", f1_score(y_test,y_pred_test))